## ***FDA Project_2 : Group - 7***

**Members:**

1) Akilesh Vishnu Mohan Raj

2) Sai Venkat Madamanchi

3) Jayanth Sai Karri

In [ ]:
#Installs Python ts2vg package to build visibility graphs from time series data.
!pip install ts2vg

In [ ]:
#Installs igraph which is a library collection for creating and manipulating graphs and analyzing networks
!pip install igraph

In [ ]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
from ts2vg import NaturalVG
from ts2vg import HorizontalVG
import numpy as np

In [ ]:
# To mount Google Drive
from google.colab import drive 
drive.mount('/content/drive')

# **Task 1** 

In [ ]:
# lists are declared for reading the required data as mentioned in the Question
activities = ['climbingdown', 'climbingup', 'walking', 'running']
body_part = ['chest', 'head']
axis = ['x', 'y', 'z']
methods = ['NVG', 'HVG']
result_task1 = pd.DataFrame(columns=['Method','Subject', 'Accelerometer_axis',
                                'Average_degree','Network_diameter',
                                'Average_path_length', 'Body_part' ,
                                'Activity'])
# to read the required activities file.
for activity in activities:
    # to read the required body parts file.
    for part in body_part:
        # to read the required number of subjects in your case it's 15.
        for i in range(1,16):
            file_loc = '/content/drive/MyDrive/Subjects/Subject'+str(i)+'/acc_'+activity+'_csv/acc_'+activity+'_'+part+'.csv'
            try:
                data = pd.read_csv(file_loc)
            # to read the file from subfolders and extract data from it
            except FileNotFoundError:
                    if (i== 4 or i== 13) and activity=="walking":
                        file_loc = '/content/drive/MyDrive/Subjects/Subject'+str(i)+'/acc_'+activity+'_csv/acc_'+activity+'_2_'+part+'.csv'
                        data = pd.read_csv(file_loc)
                    else:
                        for ij in range(1,4):
                            if ij >1:
                                file_loc = '/content/drive/MyDrive/Subjects/Subject'+str(i)+'/acc_'+activity+'_csv/'+'acc_'+activity+'_'+str(ij)+'_csv/acc_'+activity+'_'+str(ij)+'_'+part+'.csv'
                                data = pd.read_csv(file_loc)
                            else:
                                file_loc = '/content/drive/MyDrive/Subjects/Subject'+str(i)+'/acc_'+activity+'_csv/'+'acc_'+activity+'_'+str(ij)+'_csv/acc_'+activity+'_'+part+'.csv'
                                data = pd.read_csv(file_loc)
            # to read the different axis of accelerometer.
            for a in axis:
                # to get the required values through HVG AND NVG methods.
                for method in methods:
                    # NVG (natural visibility graph) method
                    if method == 'NVG':
                        ig_g = NaturalVG()
                    # HVG (horizontal visibility graph) method
                    else:
                        ig_g = HorizontalVG()
                    ig_g.build(data['attr_'+str(a)][1000:2048])
                    g = ig_g.as_igraph()
                    # Appending both method values into data frame result_task1.
                    result_task1 = result_task1.append({'Method':method, 
                                                  'Subject':i,
                                                  'Accelerometer_axis':a,
                                                  'Average_degree': np.mean(g.degree()),
                                                  'Network_diameter':g.diameter(), 
                                                  'Average_path_length':g.average_path_length(),
                                                  'Body_part':part ,
                                                  'Activity':activity}, 
                                                 ignore_index = True)
# Result data frame/table.
result_task1

In [ ]:
# The plot for average degree vs network diameter and added colour fill based on Methods. 
import seaborn as sns
sns.scatterplot(data=result_task1,x="Average_degree", y="Network_diameter",hue="Method")

In [ ]:
# Creating separate dataframes for Walking and Running
result1=result_task1.loc[(result_task1['Activity']=='walking')|(result_task1['Activity']=='running')]

# Creating separate dataframes for Climbingup and Climbingdown
result_1=result_task1.loc[(result_task1['Activity']=='climbingup')|(result_task1['Activity']=='climbingdown')]

# Function to Generate Scatter Plots for average degree vs network diameter and coloring the points according to Activities
# (doing this for each accelerometer signal and each method (HVH and NVG))
def plot_task1(result):
    # Differntiating different colours for different Activities
    colours = dict({'walking':'green', 'running':'purple','climbingup':'blue', 'climbingdown':'orange'})
    # to access each accelerometer signal,method and body part
    for axis in result['Accelerometer_axis'].unique():
        for method in result['Method'].unique():
            for part in result['Body_part'].unique():                
                plot_result = result[(result['Accelerometer_axis'] == axis) & (result['Method'] == method) & (result['Body_part'] == part)]
                title = 'Scatter plot b/w Average_degree & Network_diameter for axis: '+ str(axis)+ ' and method: ' + str(method) +' for body part: ' + str(part)
                plt.title(title)
                plt.scatter(plot_result['Average_degree'],plot_result['Network_diameter'], c = plot_result['Activity'].map(colours),label = colours)
                plt.legend()
                plt.xlabel("Average degree")
                plt.ylabel("Network diameter")
                plt.show()

# Plot for the Activities: Walking and Running
plot_task1(result1)

# Plot for the Activities: Climbingup and Climbingdown
plot_task1(result_1)

# **Scatter Plot using Seaborn FacetGrid Plot to view all charts of the Categories in a single plane.**

In [ ]:
result_11=result_task1.loc[(result_task1['Activity']=='walking')|(result_task1['Activity']=='running')]
result_HVG_head = result_11.loc[(result_11['Method']=='HVG') & (result_11['Body_part']=='head')]
result_HVG_chest = result_11.loc[(result_11['Method']=='HVG') & (result_11['Body_part']=='chest')]
result_NVG_head = result_11.loc[(result_11['Method']=='NVG') & (result_11['Body_part']=='head')]
result_NVG_chest = result_11.loc[(result_11['Method']=='NVG') & (result_11['Body_part']=='chest')]
g = sns.FacetGrid(result_HVG_head, col="Accelerometer_axis", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Average_degree", y="Network_diameter")
g.add_legend()
g.fig.suptitle('HVG Method: Scatter Plot for average degree vs network diameter based on Activity of head ')
g = sns.FacetGrid(result_HVG_chest, col="Accelerometer_axis", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Average_degree", y="Network_diameter")
g.add_legend()
g.fig.suptitle('HVG Method: Scatter Plot for average degree vs network diameter based on Activity of chest ')
print("For NVG method")
g = sns.FacetGrid(result_NVG_head, col="Accelerometer_axis", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Average_degree", y="Network_diameter")
g.add_legend()
g.fig.suptitle('NVG Method: Scatter Plot for average degree vs network diameter based on Activity of head ')
g = sns.FacetGrid(result_NVG_chest, col="Accelerometer_axis", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Average_degree", y="Network_diameter")
g.add_legend()
g.fig.suptitle('NVG Method: Scatter Plot for average degree vs network diameter based on Activity of chest ')

# **Scatter Plot using Seaborn FacetGrid Plot to view all charts of the Categories in a single plane.**

In [ ]:
result_22=result_task1.loc[(result_task1['Activity']=='climbingup')|(result_task1['Activity']=='climbingdown')]
result_HVG_head1 = result_22.loc[(result_22['Method']=='HVG') & (result_22['Body_part']=='head')]
result_HVG_chest1 = result_22.loc[(result_22['Method']=='HVG') & (result_22['Body_part']=='chest')]
result_NVG_head1 = result_22.loc[(result_22['Method']=='NVG') & (result_22['Body_part']=='head')]
result_NVG_chest1 = result_22.loc[(result_22['Method']=='NVG') & (result_22['Body_part']=='chest')]
g = sns.FacetGrid(result_HVG_head1, col="Accelerometer_axis", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Average_degree", y="Network_diameter")
g.add_legend()
g.fig.suptitle('HVG Method: Scatter Plot for average degree vs network diameter based on Activity of head ')
g = sns.FacetGrid(result_HVG_chest1, col="Accelerometer_axis", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Average_degree", y="Network_diameter")
g.add_legend()
g.fig.suptitle('HVG Method: Scatter Plot for average degree vs network diameter based on Activity of chest ')
g = sns.FacetGrid(result_NVG_head1, col="Accelerometer_axis", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Average_degree", y="Network_diameter")
g.add_legend()
g.fig.suptitle('NVG Method: Scatter Plot for average degree vs network diameter based on Activity of head ')
g = sns.FacetGrid(result_NVG_chest1, col="Accelerometer_axis", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Average_degree", y="Network_diameter")
g.add_legend()
g.fig.suptitle('NVG Method: Scatter Plot for average degree vs network diameter based on Activity of chest ')

# **TASK-2**

In [ ]:
# Function Credits: Professor Sri Radhakrishnan Git hub Repository (Professor has recommended to use the functions already build available on Github)

''' This module has essential functions supporting
fast and effective computation of permutation entropy and
its different variations.'''
import numpy as np


def s_entropy(freq_list):
    ''' This function computes the shannon entropy of a given frequency distribution.
    USAGE: shannon_entropy(freq_list)
    ARGS: freq_list = Numeric vector representing the frequency distribution
    OUTPUT: A numeric value representing shannon's entropy'''
    freq_list = [element for element in freq_list if element != 0]
    sh_entropy = 0.0
    for freq in freq_list:
        sh_entropy += freq * np.log(freq)
    sh_entropy = -sh_entropy
    return(sh_entropy)

def ordinal_patterns(ts, embdim, embdelay):
    ''' This function computes the ordinal patterns of a time series for a given embedding dimension and embedding delay.
    USAGE: ordinal_patterns(ts, embdim, embdelay)
    ARGS: ts = Numeric vector representing the time series, embdim = embedding dimension (3<=embdim<=7 prefered range), embdelay =  embdding delay
    OUPTUT: A numeric vector representing frequencies of ordinal patterns'''
    m, t = embdim, embdelay
    x = ts if isinstance(ts, np.ndarray) else np.array(ts) 

    tmp = np.zeros((x.shape[0], m))
    for i in range(m):
        tmp[:, i] = np.roll(x, i*t)
    partition = tmp[(t*m-1):, :] 
    permutation = np.argsort(partition)
    idx = _hash(permutation)

    counts = np.zeros(np.math.factorial(m))
    for i in range(counts.shape[0]):
        counts[i] = (idx == i).sum()
    return list(counts[counts != 0].astype(int))

def _hash(x):
    m, n = x.shape
    if n == 1:
        return np.zeros(m)
    return np.sum(np.apply_along_axis(lambda y: y < x[:, 0], 0, x), axis=1) * np.math.factorial(n-1) + _hash(x[:, 1:]) 
    

def p_entropy(op):
    ordinal_pat = op
    max_entropy = np.log(len(ordinal_pat))
    p = np.divide(np.array(ordinal_pat), float(sum(ordinal_pat)))
    return(s_entropy(p)/max_entropy)

def complexity(op):
    ''' This function computes the complexity of a time series defined as: Comp_JS = Q_o * JSdivergence * pe
    Q_o = Normalizing constant
    JSdivergence = Jensen-Shannon divergence
    pe = permutation entopry
    ARGS: ordinal pattern'''
    pe = p_entropy(op)
    constant1 = (0.5+((1 - 0.5)/len(op)))* np.log(0.5+((1 - 0.5)/len(op)))
    constant2 = ((1 - 0.5)/len(op))*np.log((1 - 0.5)/len(op))*(len(op) - 1)
    constant3 = 0.5*np.log(len(op))
    Q_o = -1/(constant1+constant2+constant3)

    temp_op_prob = np.divide(op, sum(op))
    temp_op_prob2 = (0.5*temp_op_prob)+(0.5*(1/len(op)))
    JSdivergence = (s_entropy(temp_op_prob2) - 0.5 * s_entropy(temp_op_prob) - 0.5 * np.log(len(op)))
    Comp_JS = Q_o * JSdivergence * pe
    return(Comp_JS)

def weighted_ordinal_patterns(ts, embdim, embdelay):
    m, t = embdim, embdelay
    x = ts if isinstance(ts, np.ndarray) else np.array(ts) 

    tmp = np.zeros((x.shape[0], m))
    for i in range(m):
        tmp[:, i] = np.roll(x, i*t)
    partition = tmp[(t*m-1):, :] 
    xm = np.mean(partition, axis=1)
    weight = np.mean((partition - xm[:, None])**2, axis=1)
    permutation = np.argsort(partition)
    idx = _hash(permutation)
    counts = np.zeros(np.math.factorial(m))
    for i in range(counts.shape[0]):
        counts[i] = sum(weight[i == idx])

    return list(counts[counts != 0]) 

In [ ]:
#To find permutation entropy and complexity for each axis and at 3 differernt signals per each activity and per each subject

# lists are declared for reading the required data as mentioned in the Question
signal_length = [1024,2048,4096]
body_part_1 = ['head','chest']
Embedded_Delays = [1,2,3]
Embedded_Dimensions = [3,4,5,6]
activities = ['climbingdown', 'climbingup', 'walking', 'running']
axis = ['x','y','z']

# Creating empty Dataframe with attributs to append the findings
result_task2 = pd.DataFrame(columns=['Subject','Accelerometer_axis', 'Signal_length', 'Dimension', 'Delay', 'Permutation_entropy','Complexity','Body_part', 'Activity'])

# to read the required activities file
for activity in activities:
    # to read the required body parts file
    for body in body_part_1:
        # to read the required number of subjects in your case it's 15
        for i in range(1,16):
            file_loc='/content/drive/MyDrive/Subjects/Subject'+str(i)+'/acc_'+activity+'_csv/acc_'+activity+'_'+body+'.csv'
            try:
                data = pd.read_csv(file_loc)
            # to read the file from subfolders and extract data from it
            except FileNotFoundError:
                if (i== 4 or i== 13) and activity=="walking":
                    file_loc = '/content/drive/MyDrive/Subjects/Subject'+str(i)+'/acc_'+activity+'_csv/acc_'+activity+'_2_'+body+'.csv'
                    data = pd.read_csv(file_loc)
                else:
                    for ij in range(1,4):
                        if ij >1:
                            file_loc = '/content/drive/MyDrive/Subjects/Subject'+str(i)+'/acc_'+activity+'_csv/'+'acc_'+activity+'_'+str(ij)+'_csv/acc_'+activity+'_'+str(ij)+'_'+body+'.csv'
                            data = pd.read_csv(file_loc)
                        else:
                            file_loc = '/content/drive/MyDrive/Subjects/Subject'+str(i)+'/acc_'+activity+'_csv/'+'acc_'+activity+'_'+str(ij)+'_csv/acc_'+activity+'_'+body+'.csv'
                            data = pd.read_csv(file_loc)
            # To read different signal lengths
            for length in signal_length:
                data1 = data[1000:(1000+length+1)]
                # to read the different axis of accelerometer
                for ax in axis:
                    # to read the Embedded Dimension Values
                    for dimension in Embedded_Dimensions:
                        # to read the Embedded Delay Values
                        for delay in Embedded_Delays:
                            op = ordinal_patterns(data1['attr_'+str(ax)],dimension,delay)
                            npe = p_entropy(op)
                            ncomp = complexity(op)
                            result_task2 = result_task2.append({'Subject':i,'Accelerometer_axis':ax, 'Signal_length':length, 'Dimension':dimension, 'Delay':delay, 'Permutation_entropy':npe,'Complexity':ncomp,'Body_part':body, 'Activity':activity}, ignore_index=True)

# Result data frame/table.
result_task2

In [ ]:
# Generating Scatter plots based on the below conditions as per the Task Requirement:

#1) Signal length =4096
#2) Embedded delay = 1
#3) Embedded dimension = 3, 4, 5, 6
#4) All three accelerometer directions

# Creating Dataframe for the Activities walking and running and Embedded delay = 1
results_2 = result_task2.loc[(result_task2['Activity']=='walking')|(result_task2['Activity']=='running')]
results2 = results_2.loc[(results_2['Delay']== 1)]

# Creating Dataframe for the Activities climbingup and climbingdown and Embedded delay = 1
results_3 = result_task2.loc[(result_task2['Activity']=='climbingup')|(result_task2['Activity']=='climbingdown')]
results3 = results_3.loc[(results_3['Delay']== 1)]

# Function to Generate Scatter Plots for permutation entropy vs complexity and coloring the points according to Activities
def plot(result):
    colours = dict({'walking':'orange', 'running':'purple', 'climbingup':'green', 'climbingdown':'blue'})
    for axis in result['Accelerometer_axis'].unique():
        for dimension in result['Dimension'].unique():
            for part in result['Body_part'].unique():
                plot_result = result[(result['Accelerometer_axis'] == axis) & (result['Dimension'] == dimension ) & (result['Body_part'] == part) & (result['Signal_length']== 4096)]
                title = 'Scatter plot b/w Permutation_entropy & Network_diameter for axis: '+ str(axis)+ ' and Dimension: ' + str(dimension) +' for body part: ' + str(part)
                plt.title(title)
                plt.scatter(plot_result['Permutation_entropy'],plot_result['Complexity'], c = plot_result['Activity'].map(colours),label= colours)
                plt.legend()
                plt.xlabel("Permutation_entropy")
                plt.ylabel("Complexity")
                plt.show()

# Plot for the Activities: Walking and Running
plot(results2)

# Plot for the Activities: Climbingup and Climbingdown
plot(results3)

# **Scatter Plot using Seaborn FacetGrid Plot to view all charts of the Categories in a single plane.**

In [ ]:
result_x= results2.loc[(results2['Accelerometer_axis']=='x') & (results2['Signal_length']==4096) & (results2['Body_part']=='head')]
g = sns.FacetGrid(result_x, col="Dimension", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Permutation_entropy", y="Complexity")
g.fig.suptitle('Scatter Plot for permutation entropy vs complexity based on Activity of head in x axis ')
g.add_legend()
result_y= results2.loc[(results2['Accelerometer_axis']=='y') & (results2['Signal_length']==4096) & (results2['Body_part']=='head')]
g = sns.FacetGrid(result_y, col="Dimension", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Permutation_entropy", y="Complexity")
g.fig.suptitle('Scatter Plot for permutation entropy vs complexity based on Activity of head in y axis')
g.add_legend()
result_z= results2.loc[(results2['Accelerometer_axis']=='z') & (results2['Signal_length']==4096) & (results2['Body_part']=='head')]
g = sns.FacetGrid(result_z, col="Dimension", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Permutation_entropy", y="Complexity")
g.fig.suptitle('Scatter Plot for permutation entropy vs complexity based on Activity of head in z axis')
g.add_legend()

# **Scatter Plot using Seaborn FacetGrid Plot to view all charts of the Categories in a single plane.**

In [ ]:
result_x1= results2.loc[(results2['Accelerometer_axis']=='x') & (results2['Signal_length']==4096) & (results2['Body_part']=='chest')]
g = sns.FacetGrid(result_x1, col="Dimension", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Permutation_entropy", y="Complexity")
g.fig.suptitle('Scatter Plot for permutation entropy vs complexity based on Activity of chest in x axis ')
g.add_legend()
result_y1= results2.loc[(results2['Accelerometer_axis']=='y') & (results2['Signal_length']==4096) & (results2['Body_part']=='chest')]
g = sns.FacetGrid(result_y1, col="Dimension", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Permutation_entropy", y="Complexity")
g.fig.suptitle('Scatter Plot for permutation entropy vs complexity based on Activity of chest in y axis')
g.add_legend()
result_z1= results2.loc[(results2['Accelerometer_axis']=='z') & (results2['Signal_length']==4096) & (results2['Body_part']=='chest')]
g = sns.FacetGrid(result_z1, col="Dimension", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Permutation_entropy", y="Complexity")
g.fig.suptitle('Scatter Plot for permutation entropy vs complexity based on Activity of chest in z axis')
g.add_legend()

# **Scatter Plot using Seaborn FacetGrid Plot to view all charts of the Categories in a single plane.**

In [ ]:
result_x2= results3.loc[(results3['Accelerometer_axis']=='x') & (results3['Signal_length']==4096) & (results3['Body_part']=='head')]
g = sns.FacetGrid(result_x2, col="Dimension", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Permutation_entropy", y="Complexity")
g.fig.suptitle('Scatter Plot for permutation entropy vs complexity based on Activity of head in x axis ')
g.add_legend()
result_y2= results3.loc[(results3['Accelerometer_axis']=='y') & (results3['Signal_length']==4096) & (results3['Body_part']=='head')]
g = sns.FacetGrid(result_y2, col="Dimension", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Permutation_entropy", y="Complexity")
g.fig.suptitle('Scatter Plot for permutation entropy vs complexity based on Activity of head in y axis')
g.add_legend()
result_z2= results3.loc[(results3['Accelerometer_axis']=='z') & (results3['Signal_length']==4096) & (results3['Body_part']=='head')]
g = sns.FacetGrid(result_z2, col="Dimension", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Permutation_entropy", y="Complexity")
g.fig.suptitle('Scatter Plot for permutation entropy vs complexity based on Activity of head in z axis')
g.add_legend()

# **Scatter Plot using Seaborn FacetGrid Plot to view all charts of the Categories in a single plane.**

In [ ]:
result_x3= results3.loc[(results3['Accelerometer_axis']=='x') & (results3['Signal_length']==4096) & (results3['Body_part']=='chest')]
g = sns.FacetGrid(result_x1, col="Dimension", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Permutation_entropy", y="Complexity")
g.fig.suptitle('Scatter Plot for permutation entropy vs complexity based on Activity of chest in x axis ')
g.add_legend()
result_y3= results3.loc[(results3['Accelerometer_axis']=='y') & (results3['Signal_length']==4096) & (results3['Body_part']=='chest')]
g = sns.FacetGrid(result_y3, col="Dimension", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Permutation_entropy", y="Complexity")
g.fig.suptitle('Scatter Plot for permutation entropy vs complexity based on Activity of chest in y axis')
g.add_legend()
result_z3= results3.loc[(results3['Accelerometer_axis']=='z') & (results3['Signal_length']==4096) & (results3['Body_part']=='chest')]
g = sns.FacetGrid(result_z3, col="Dimension", hue="Activity")
g.map_dataframe(sns.scatterplot, x="Permutation_entropy", y="Complexity")
g.fig.suptitle('Scatter Plot for permutation entropy vs complexity based on Activity of chest in z axis')
g.add_legend()